# We're scraping

Probably going to just scrape it all. 

See [baseball-scraper](https://pypi.org/project/baseball-scraper/) for examples.

In [1]:
from baseball_scraper import statcast
from baseball_scraper import playerid_lookup

import pandas as pd 
import datetime 

In [2]:
##What happens if I ask for a date on which no games are played... 
##Like I write a function and uh-oh, all-star break included...

##Looking for a date pre statcast gives a table with lots of NaN values. 
#pre_stat_scrape = statcast(start_dt='2010-08-08', verbose=True)

##Looking for a date on which no games were played gives a dataframe of shape 0x90
#january_scrape = statcast(start_dt='2018-01-01', verbose=True)




'2018-01-01'

In [3]:
start = datetime.date(2020,7,23).strftime("%Y-%m-%d")
end = datetime.date(2020,8,18).strftime("%Y-%m-%d")##This is the world's third hardest way to write '2020-08-18', but you're maybe using it later.

This is a large query, it may take a moment to complete
Completed sub-query from 2020-07-23 to 2020-07-28
Completed sub-query from 2020-07-29 to 2020-08-03
Completed sub-query from 2020-08-04 to 2020-08-09
Completed sub-query from 2020-08-10 to 2020-08-15
Completed sub-query from 2020-08-16 to 2020-08-18


In [7]:
## Scrapes time slices based on a date range. Returns a list of successfully scraped time slices.

def big_scrape(start,end):
    '''
    Returns a list of all the successfull scraped time slices in range start, end. 
    
    If an exception     is raised, prints a brief report, stops scraping, and returns
    all the successfully scraped slices.
    '''
    scrapes = [] ##
    while True:
        try:
            temp = statcast(start.strftime("%Y-%m-%d"), (start+datetime.timedelta(days=4)).strftime("%Y-%m-%d"))
        except ParserError:
            print("ParserError failure at {}".format(start.strftime("%Y-%m-%d")))
            break
        except:
            print("Unspecified failure at {}".format(start.strftime("%Y-%m-%d")))
            break
        scrapes.append(temp)
        start = start+datetime.timedelta(days=5)
        if start > end:
            break
    return scrapes

2020-08-19 10:32:58.162340
2020-08-19 10:32:58.162670


In [9]:
## Creates all the .csv files.

for dt in [2019,2018,2017,2016]:
    start = datetime.date(dt,3,20)
    end = datetime.date(dt,10,1)
    scrape = big_scrape(start,end)
    pd.concat(scrape).to_csv('data/statcast_dumps/statcast'+str(dt)+'.csv')
    print("Finished with statcast year {}.".format(dt))
    del scrape

Finished with statcast year 2019.
Finished with statcast year 2018.
Finished with statcast year 2017.
Finished with statcast year 2016.
